In [1]:
%matplotlib inline
# Importam biblioteci utile pentru manipularea datelor si AI
import pandas as pd   # pentru a lucra cu tabele (date structurate)
import numpy as np    # pentru calcule matematice
import tensorflow as tf  # biblioteca principala pentru AI

import os

# Importam functii din Keras pentru a procesa text si a construi modelul AI
from tensorflow.keras.preprocessing.text import Tokenizer   # transforma textul in numere
from keras.preprocessing.sequence import pad_sequences      # face ca toate textele sa aiba aceeasi lungime
from keras.layers import Dense, Embedding, LSTM             # tipuri de "straturi" din reteaua neuronala
from keras.optimizers import Adam                           # metoda de invatare
from keras.layers import Input
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint                 # pentru a salva modelul in timpul antrenarii
import keras.backend as K
import keras

# Biblioteci pentru procesarea limbajului natural (text)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')  # descarcă lista de cuvinte comune
nltk.download('wordnet')
stop_words = stopwords.words('german')   # folosim lista de cuvinte comune in germana (care nu ajuta modelul)
import re
pd.set_option('display.precision',20)

print("Tensorflow version " + tf.__version__)


Tensorflow version 2.19.0


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\const\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\const\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# ------------------------------------------------------------
# Incarcam datele (texte de antrenament, test si validare)
# ------------------------------------------------------------
# training.txt contine exemple folosite pentru invatarea AI-ului
df_train = pd.read_csv('dataset/training.txt', names=['id', 'lat','long','tweet'], header=None)
df_train.head()

,id,lat,long,tweet
0,119165,51.81006711409395393275,10.19133093525180200345,"Seit d Vase: ""Wenn ich kaputt gang, bringt das..."
1,100377,51.91818791946306532736,10.59924460431654580361,Haha bin au w isch der amig au so richtig lang...
2,109550,52.71107382550334818916,9.98737410071942477430,isch d hiltl dachterrasse amne samstig viel bs...
3,111440,52.38671140939595716191,11.70061151079136863018,Ich fühle mich wie die Weimarer Republik... .....
4,116670,52.31463087248320675826,9.70183453237410375891,Eui liebschte Lunchidee zum Mitneh? 😬 En Grill...


In [4]:
# test.txt contine exemple noi pe care le vom folosi pentru a verifica modelul
df_test = pd.read_csv('dataset/test.txt', names=['id','tweet'], header=None)
df_test.head()

,id,tweet
0,300121,👩min vibi funktionkert nöd... 👧hesch d'batteri...
1,302441,Ich: Also langsam söti scho schlafe Au Ich: He...
2,300266,Hez hie ou lüt wo dr ganz tag ine biudschirm m...
3,300911,je neui wohnig 😎 neua job 😎 eigeni kuchi 😎 abe...
4,302681,Schön wies grad chunt cho hagle u du nid d müg...


In [5]:
# validation.txt este folosit pentru a evalua modelul in timpul antrenarii
df_valid = pd.read_csv('dataset/validation.txt', names=['id', 'lat','long','tweet'], header=None)
df_valid.head()

,id,lat,long,tweet
0,203001,52.09838926174496265276,10.35449640287770023406,wenn mer anere Party bi Kollege en neue Bro fi...
1,200313,51.62986577181206371279,7.72345323741007305784,Heii guetä wuchestart gha? Jo wunderbar und du...
2,201966,52.24255033557045635462,8.96758992805755283939,Wieso ned? De werds eim emel ned langwiilig. O...
3,201123,52.24255033557045635462,8.96758992805755283939,Hani welle vo zueri uf lausanne denn hetts e d...
4,200374,51.70194630872484253814,8.41690647482014497882,Mir isch die Applikation plagiert worde as Dis...


In [6]:
# ------------------------------------------------------------
# Functie care curata textul (elimina caractere inutile)
# ------------------------------------------------------------
def clean_text(input_text):
    text = re.sub(r'\W',' ', input_text) # Elimină toate caracterele non-cuvinte
    text = re.sub(r'\d+',' ', text) # Elimină toate cifrele
    text = input_text.lower() # Convertim textul în litere mici
    text = re.sub(r'\s+[a-z]\s+',' ', text) # Elimină toate literele singure
    text = re.sub(r'^\s+','', text) # Elimină spațiile de la începutul textului
    text = re.sub(r'\s+$','', text) # Elimină spațiile de la sfârșitul textului
    text = re.sub(r'\s+',' ', text) # Elimină toate spațiile multiple
    text = text.split(' ') # Împarte cuvintele în tokeni
    text = [word for word in text if word not in stop_words] # Elimină cuvintele comune
    text = [WordNetLemmatizer().lemmatize(word) for word in text] # Aduce cuvintele la forma de bază
    text = ' '.join(text)

    return text

In [7]:
df_train['tweet'] = df_train.tweet.apply(lambda x: clean_text(x))

In [8]:
df_test['tweet'] = df_test.tweet.apply(lambda x: clean_text(x))

In [9]:
df_valid['tweet'] = df_valid.tweet.apply(lambda x: clean_text(x))

In [10]:
# Pregătirea datelor pentru model
X = df_train['tweet']  # Textul de antrenament (caracteristicile)
y_lat = df_train['lat']/100  # Latitudinea țintă (normalizată)
y_long = df_train['long']/100  # Longitudinea țintă (normalizată)

X_valid = df_valid['tweet']  # Date de validare
y_lat_valid = df_valid['lat']/100
y_long_valid = df_valid['long']/100

test = df_test['tweet']  # Date de test

In [11]:
# Analiză a vocabularului
string=''
for str in df_train['tweet']:
  string+=str+' '
spl = string.split()
print(len(spl))  # Numărul total de cuvinte
myset = set(spl)
print(len(myset))  # Numărul de cuvinte distincte

979525
172542


In [12]:
# Crearea tokenizer-ului (converteste cuvintele în numere)
num_words = 2000  # Vom lua în considerare doar primele 2000 de cuvinte cele mai frecvente
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df_train['tweet'].values)  # Învață vocabularul din datele de antrenament

In [13]:
# Transformarea textului în secvențe numerice și padding
X = tokenizer.texts_to_sequences(df_train['tweet'].values)  # Converteste textul în numere
X = pad_sequences(X, maxlen=num_words)  # Face ca toate secvențele să aibă aceeași lungime

X_valid = tokenizer.texts_to_sequences(df_valid['tweet'].values)
X_valid = pad_sequences(X_valid, maxlen=num_words)

test = tokenizer.texts_to_sequences(df_test['tweet'].values)
test = pad_sequences(test, maxlen=num_words)

In [14]:
# Setarea parametrilor modelului
embed_dim = 128  # Dimensiunea vectorilor de embedding
lstm_out = 192   # Numărul de unități LSTM

In [15]:
# Definirea arhitecturii modelului
from keras.layers import Dropout
def create_model():
    # Stratul de intrare - primește secvențe de numere de lungime fixă (num_words)
    input_tensor = Input(shape=(num_words, ))  # Stratul de intrare

    # ------------------------------------------------------------
    # ARHITECTURA PENTRU PRIMUL OUTPUT (LATITUDINE)
    # ------------------------------------------------------------
  
    # Strat Embedding: transformă indicii cuvintelor în vectori densi (reprezentări numerice)
    # num_words = 2000 (vocabularul), embed_dim = 128 (dimensiunea vectorilor)
    hidden_1=Embedding(num_words, embed_dim, input_length = X.shape[1])(input_tensor)

    # Strat LSTM: procesează secvența de vectori pentru a învăța dependențe temporale
    # lstm_out = 192 (numărul de unități LSTM), dropout pentru a preveni overfitting
    hidden_1=LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2)(hidden_1)
  
    # Strat Dense (fully connected): 500 de neuroni cu activare ReLU
    # Acest strat extrage caracteristici complexe din output-ul LSTM
    hidden_1=Dense(500, activation='relu')(hidden_1)
  
    # Încă un strat Dense pentru a adăuga capacitate de învățare
    hidden_1=Dense(500, activation='relu')(hidden_1)
  
    # Strat de output pentru latitudine: un singur neuron (valoare continuă)
    # Activare ReLU pentru a asigura valori pozitive, fără bias, initializare normală
    output_tensor_1 = Dense(1, activation='relu', use_bias=False, kernel_initializer='normal')(hidden_1)
  
    # ------------------------------------------------------------
    # ARHITECTURA PENTRU AL DOILEA OUTPUT (LONGITUDINE)
    # ------------------------------------------------------------
    
    # Folosim output-ul de la latitudine ca input pentru longitudine
    # Această arhitectură sugerează că există o relație între cele două coordonate
    hidden_2=Dense(500, activation='relu')(output_tensor_1)
  
    # Straturi adiționale Dense pentru a procesa informația
    hidden_2=Dense(500, activation='relu')(hidden_2)
    hidden_2=Dense(500, activation='relu')(hidden_2)
  
    # Strat de output pentru longitudine: un singur neuron (valoare continuă)
    # Fără funcție de activare specifică (output liniar) pentru valori pozitive și negative
    output_tensor_2 = Dense(1)(hidden_2)
  
    # ------------------------------------------------------------
    # CONSTRUIREA MODELULUI FINAL
    # ------------------------------------------------------------
    
    # Creăm modelul cu un input și două output-uri (latitudine și longitudine)
    model = Model(input_tensor, [output_tensor_1,output_tensor_2])

    # Optimizator Adam: algoritm eficient pentru ajustarea ponderilor
    optimizer = Adam()
  
    # Compilarea modelului: specificăm funcțiile de loss pentru fiecare output
    # Mean Absolute Error (MAE) măsoară eroarea medie absolută între predicții și valori reale
    model.compile(loss=['mean_absolute_error','mean_absolute_error'],optimizer=optimizer)
    return model

In [16]:
# Crearea modelului
model = create_model()

C:\Users\const\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
# Setup pentru salvarea modelului
filepath='cp_keras.h5'
dirpath=os.path.dirname(filepath)
checkpoint=ModelCheckpoint(filepath,monitor='val_loss',verbose=1,save_best_only=True)

In [18]:
# Antrenăm modelul keras pe dataset
model.fit(X, [y_lat,y_long], epochs=1, validation_data=(X_valid, [y_long_valid,y_lat_valid]),callbacks=[checkpoint],shuffle=True)

706/706 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - dense_2_loss: 0.0327 - dense_6_loss: 0.0158 - loss: 0.0485
Epoch 1: val_loss improved from None to 0.84318, saving model to cp_keras.h5


706/706 ━━━━━━━━━━━━━━━━━━━━ 2867s 4s/step - dense_2_loss: 0.0137 - dense_6_loss: 0.0122 - loss: 0.0259 - val_dense_2_loss: 0.4220 - val_dense_6_loss: 0.4213 - val_loss: 0.8432


In [19]:
# Încărcarea modelului salvat
model = tf.keras.models.load_model(filepath)

In [20]:
# summarize layers
print(model.summary())  # Arată structura modelului și numărul de parametri

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 2000, 128)      │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 192)            │       246,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │        96,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 500)            │         1,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,352,531 (5.16 MB)

 Trainable params: 1,352,529 (5.16 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None


In [24]:
# Vizualizarea arhitecturii modelului (necesită pydot)
from keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [25]:
# Realizarea de predicții pe datele de validare
predictions_valid = model.predict(X_valid)

96/96 ━━━━━━━━━━━━━━━━━━━━ 50s 520ms/step


In [26]:
# Adăugarea predicțiilor în dataframe-ul de validare
df_valid['predicted_lat']=predictions_valid[0].max(axis=1)

In [27]:
df_valid['predicted_long']=predictions_valid[1].max(axis=1)

In [28]:
# Evaluarea modelului pe datele de validare
print(model.evaluate(X_valid,
                     [y_lat_valid,y_long_valid],
                     verbose=False))  # Afișează eroarea medie absolută

[0.016810571774840355, 0.006307543721050024, 0.010465609841048717]


In [29]:
df_valid  # Arată datele originale și predicțiile

,id,lat,long,tweet,predicted_lat,predicted_long
0,203001,52.09838926174496265276,10.35449640287770023406,mer anere party bi kollege en neue bro findt h...,0.52146583795547485352,0.09638313204050064087
1,200313,51.62986577181206371279,7.72345323741007305784,heii guetä wuchestart gha? jo wunderbar du? ja...,0.51711761951446533203,0.09637607634067535400
2,201966,52.24255033557045635462,8.96758992805755283939,wieso ned? de werds eim emel ned langwiilig. o...,0.51607179641723632812,0.09637437760829925537
3,201123,52.24255033557045635462,8.96758992805755283939,hani welle vo zueri uf lausanne hetts durchsag...,0.51934522390365600586,0.09637968987226486206
4,200374,51.70194630872484253814,8.41690647482014497882,isch applikation plagiert worde a diskussionsf...,0.52085357904434204102,0.09638213366270065308
...,...,...,...,...,...,...
3039,200754,52.62047945205478072239,10.29454545454545666416,git hüt schöns mache? 😄 (elei haha) uf em sofa...,0.51792967319488525391,0.09637739509344100952
3040,201772,52.84116438356164735524,7.80318181818181955123,grad sm vo salt beko: unglaubliches angebot: g...,0.51996272802352905273,0.09638069570064544678
3041,201431,51.88486301369860598243,8.18977272727272520569,lauft chestehouz uf em hoger obe?🤔 voll schien...,0.51891160011291503906,0.09637898951768875122
3042,202066,51.77452054794520108771,9.65022727272727287584,glaub scho chli en psycho liebs go jogge zgah ...,0.51548910140991210938,0.09637343883514404297


In [30]:
# Realizarea de predicții pe datele de test
predictions_test = model.predict(test)

99/99 ━━━━━━━━━━━━━━━━━━━━ 47s 477ms/step


In [31]:
# Pregătirea rezultatelor pentru export
df_test['lat']=predictions_test[0].max(axis=1)
df_test['long']=predictions_test[1].max(axis=1)

In [32]:
df_test=df_test.drop('tweet', axis=1)  # Elimină coloana cu text

In [33]:
print(df_test)

          id                     lat                    long
0     300121  0.51732438802719116211  0.09637641906738281250
1     302441  0.51114243268966674805  0.09636639058589935303
2     300266  0.50165534019470214844  0.09635099768638610840
3     300911  0.51744657754898071289  0.09637661278247833252
4     302681  0.51230037212371826172  0.09636826813220977783
...      ...                     ...                     ...
3133  300151  0.51738721132278442383  0.09637651592493057251
3134  300302  0.51961028575897216797  0.09638011455535888672
3135  301963  0.51894670724868774414  0.09637904167175292969
3136  302180  0.50940078496932983398  0.09636355936527252197
3137  301697  0.52185487747192382812  0.09638375788927078247

[3138 rows x 3 columns]


In [ ]:
df_test.to_csv('dataset/rezultatKerasModel1.txt', index=False, decimal='.', sep=',', float_format='%.20f')